In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from icevision import show_records
from icevision.imports import *
from icevision.utils import *
from icevision.data import *
from icevision.metrics.metric import *
from icevision.models.utils import get_dataloaders
from fastai.callback.tracker import SaveModelCallback
from fastai.callback.tensorboard import TensorBoardCallback
from icevision.engines.fastai import convert_dataloaders_to_fastai
from icevision.metrics import SimpleConfusionMatrix # make sure you have the SkyTruth fork of icevision installed
import torch

In [ ]:
from ceruleanml.learner_config import (
    run_list,
    classes_to_keep,
    model_type,
    model,
    get_tfms,
    wd,
    record_collection_train,
    record_collection_val,
    record_collection_test,
    model_name,
    num_workers,
)
run_list

# Training Parameters

In [ ]:
bs_d ={512:8, 256:16, 224:16, 128:32, 64:64} # Batch Size for each image size
lr_d = {512:1e-3, 256:1e-3, 224:1e-3, 128:1e-3, 64:1e-3} # Learning Rate for each image size
mins_d = {512:1.4, 256:2, 224:1.3, 128:1.5, 64:1.2}

In [ ]:
_, dls = get_dataloaders(model_type, [record_collection_train, record_collection_val], get_tfms(), batch_size=bs_d[run_list[0][0]])

metrics = []

learner = model_type.fastai.learner(dls=dls, model=model, cbs=[SaveModelCallback(min_delta=.01)], metrics=metrics, wd=wd)
learner.dls = convert_dataloaders_to_fastai(dls=dls)

In [ ]:
print(classes_to_keep)
print(model_name+"_intermediate")

In [ ]:
from datetime import datetime
from ceruleanml.inference import save_icevision_model_state_dict_and_tracing

def export_scripted_model(learner, model_name):
    dateTimeObj = datetime.now()
    timestampStr = dateTimeObj.strftime("%Y_%m_%d_%H_%M_%S")
    experiment_dir =  Path(f'/root/experiments/cv2/{timestampStr}_{model_name}_maskrcnn/')
    experiment_dir.mkdir(exist_ok=True)
    print(experiment_dir)
    save_template = "model.pt"
    save_icevision_model_state_dict_and_tracing(learner, save_template, experiment_dir)

In [ ]:
print("Exporting last saved model")
learner.load("model")
export_scripted_model(learner, model_name+"_intermediate")